# Parameter sweep

In [ ]:
using ProgressMeter, CSV, DataFrames

function run_ODE_Solve(ps, ps_init; model = "pedagogical", folder = "parameter screen")
    # function to run one simulation with parameters ps and initial conditions ps_init
    if model == "pedagogical" # Pedagogical model is where the hill function is replaced by step functions, not presented in this code
        D, a, gammaL, L0, L1, b, I, rho, S_on, S_len, l_mul, t_mul = ps
        ps_temp = D, a, gammaL, L0, L1, b, I, rho, S_on, S_len
    elseif model == "practical"
        D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_len, l_mul, t_mul = ps
        ps_temp = D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_len
    end
    filename = string("Son-", round(S_on,sigdigits = 3),"_a-",round(a,sigdigits = 3),"_L1a-", round(L0/(a*rho),sigdigits = 3),
        "_L1L2-", round(L0/L1,sigdigits=3), "_R1b-", round(I/b,sigdigits=3))

    filepath = joinpath(folder, string(filename,".csv") )
    params_filepath = joinpath(folder, string(filename,"_params.json"))
    if isfile(filepath)
        return
    end

    function div_2d3d(N_r, N_z, r_max, z_max)
        rd = range(0, stop=r_max, length=N_r)
        zd = range(0, stop=z_max, length=N_z)
        dr = step(rd)
        dz = step(zd)
        translate_2d3d_to_1d(ir, iz) = (iz - 1) * N_r + ir
        ir_indices = Int[]
        jc_indices = Int[]
        data_values = Float64[]
        for ir in 1:N_r
            r = rd[ir]
            for iz in 1:N_z
                center_idx = translate_2d3d_to_1d(ir, iz)
                center_val = 0.
                # Radial parts
                if ir > 1 # Left neighbor
                    push!(ir_indices, center_idx)
                    push!(jc_indices, translate_2d3d_to_1d(ir - 1, iz))
                    push!(data_values, 1/dr^2 - 1/(2*dr*r))
                end
                if ir < N_r # Right neighbor
                    push!(ir_indices, center_idx)
                    push!(jc_indices, translate_2d3d_to_1d(ir + 1, iz))
                    if ir != 1
                        push!(data_values, 1/dr^2 + 1/(2*dr*r))
                    else
                        push!(data_values, 1/dr^2)
                    end
                    # push!(data_values, 1/dr^2 + 1/(2*dr*r))
                end
                if ir != 1
                    center_val -= 1/(dr^2)
                end
    
                # Axial parts
                if iz > 1 # Below neighbor
                    push!(ir_indices, center_idx)
                    push!(jc_indices, translate_2d3d_to_1d(ir, iz - 1))
                    push!(data_values, 1/dz^2)
                end
                if iz < N_z # Above neighbor
                    push!(ir_indices, center_idx)
                    push!(jc_indices, translate_2d3d_to_1d(ir, iz + 1))
                    push!(data_values, 1/dz^2)
                end
    
                # Center position, adjusted for radial dependence
                center_val -= (1/dr^2 + 2/dz^2) 
                push!(ir_indices, center_idx)
                push!(jc_indices, center_idx)
                push!(data_values, center_val)
            end
        end
    
        # Construct the sparse matrix
        mat_div = sparse(ir_indices, jc_indices, data_values)
        return mat_div
    end
    #Env setup: 
    r_limit = 50*l_mul # simulation maximum radius 500 microns
    z_limit = 5*l_mul # simulation maximum height 50 microns
    r_inputsource = 1*l_mul # initial source size 10 microns
    H = 0.1*l_mul #cell height 1 micron
    N_r = 500 # number of radial grid points
    N_z = 125 # number of axial grid points
    mat_div = div_2d3d(N_r,N_z,r_limit,z_limit);
    rd_LTB4 = range(0,stop = r_limit,length = N_r)
    zd_LTB4 = range(0,stop = z_limit, length = N_z)

    dr = step(rd_LTB4)
    dz = step(zd_LTB4)

    zs = repeat(zd_LTB4,inner = N_r)
    ir_max = ceil(Int,r_inputsource/dr)+1
    iz_min = floor(Int,(z_limit/2 - H/2)/dz)
    iz_max = floor(Int,(z_limit/2 + H/2)/dz)+1

    delta(z,H) = 1/(2H)*sqrt(2/pi)*exp(-z^2 / (2*H^2)); #1D Gaussian distribution to mimic cell distribution over z axis
    deltaz_vec = delta.(zs .- z_limit/2,H)
    deltar(r,R) = 1/(R^2)*exp(-r^2 /(R^2)); #2D Gaussian distribution in polar coordinates
    source_T(t,S0,S_l) = (t >= 0)*(t <= S_l)*S0;

    translate_2d3d_to_1d(ir,iz,N_r) = (iz - 1)*N_r + ir;

    function gill_f(N,K,h)
        if h > 0
            return K^h/(N^h + K^h)
        else
            return 1*(N < K)
        end
    end

    function hill_f(N,K,h)
        if h > 0
            return N^h /(N^h + K^h)
        else
            return 1*(N > K)
        end
    end


    function LTB4_relay_gdecay_practical_2d3d_diff(dX,X,ps,t)
        #PDE model
        D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_len = ps 
        # D = D/(dx^2)
        @inbounds dX[:,1] =  D*mat_div*X[:,1] .+ a*rho.*gill_f.(X[:,2],I,3).*hill_f.(X[:,1],L0,3).*deltaz_vec*2 .- gammaL*X[:,1]
        @inbounds dX[:,2] = b*hill_f.(X[:,1],L1,5) - gammaR*X[:,2]
        if S_on > 0. && t<S_len
            @inbounds for ir in 1:ir_max
            for iz in iz_min:iz_max
                dX[translate_2d3d_to_1d(ir,iz,N_r),1] += S_on
            end
            end
        end
    end


    function init_LTB4_inhib_2d3d(rd,zd,S_int,r_source , R0)
        #initialization function
        N_r = length(rd)
        N_z = length(zd)
        source_Trz(r,z,t,S0) = deltar(r,r_source)*delta(z - z_limit/2,H)*(t >= 0)*S0; #source in the center
        u = zeros(N_r*N_z,2)
        for I in CartesianIndices((N_r, N_z))
            r = rd[I[1]]
            z = zd[I[2]]
            u[translate_2d3d_to_1d(I[1],I[2],N_r),1] = source_Trz(r,z,1.,S_int)
            u[translate_2d3d_to_1d(I[1],I[2],N_r),2] = R0
        end
        u
    end

    function wave_front_r(X, rd ,iz, S_cri)
        #Find the wave front position, used for analyzing the solution
        N_r = length(rd)
        dr = step(rd)
        ixs = translate_2d3d_to_1d.( 1:N_r , iz, N_r)
        # ixs = ixs - N/2
        ix = finDast(X[ixs].> S_cri)
        if !isnothing(ix)
            ix = ix
            x = ix*dr
        else
            x = 0
        end
        x
    end
  
  
    #Step size
    dr = step(rd_LTB4)
    dz = step(zd_LTB4)
    if model == "pedagogical" 
        D, a, gammaL, L0, L1, b, I, rho, S_on, S_len = ps
    elseif model == "practical"
        D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_len = ps
    end
  
  
    #Setting initatal conditions and building model space: 
    r_source, s_init = ps_init #Initial source size & amplitude
    #Setting initatal conditions and building model space: 
    X02d3d = init_LTB4_inhib_2d3d(rd_LTB4, zd_LTB4, s_init, r_source, 0.0);
    dX02d3d = copy(X02d3d);
  
    #Test model ok, then build sparsity matrix: 
    #LTB4_relay_gdecay_NADPH_2d3d_diff(dX02d3d_simple,X02d3d,ps_relay_w_decay_L1_inhib,1)
    if model == "pedagogical"
        jac_sparsity_2d3d_inhib_relay_with_decay = Symbolics.jacobian_sparsity((dX,X)->LTB4_relay_gdecay_pedagogical_2d3d_diff(dX,X,ps_temp,1),dX02d3d,X02d3d);
    elseif model == "practical"
        jac_sparsity_2d3d_inhib_relay_with_decay = Symbolics.jacobian_sparsity((dX,X)->LTB4_relay_gdecay_practical_2d3d_diff(dX,X,ps_temp,1),dX02d3d,X02d3d);
    end

    t_end = 5.0*t_mul
    savedt = 0.01*t_mul 
    #Setup function as ODE function, give sparsity matrix: 
    if model == "pedagogical"
        f3 = ODEFunction(LTB4_relay_gdecay_pedagogical_2d3d_diff;jac_prototype=float.(jac_sparsity_2d3d_inhib_relay_with_decay));
    elseif model == "practical"
        f3 = ODEFunction(LTB4_relay_gdecay_practical_2d3d_diff;jac_prototype=float.(jac_sparsity_2d3d_inhib_relay_with_decay));
    end
    LTB4_relay_inhib_w_decay_2d3d_sparse = ODEProblem(f3,X02d3d,(0.0,t_end),ps_temp); #Setup ODE problem:
    LTB4_relay_inhib_w_decay_2d3d_sol = solve(LTB4_relay_inhib_w_decay_2d3d_sparse, Kvaerno5(), saveat = savedt); #Solve ODE problem using auto switching with rosenbrock23, see https://docs.scigammaL.ai/DiffEqDocs/stable/solvers/ode_solve/
    


    
    # Create the folder if it does not exist
    if !isdir(folder)
        mkdir(folder)
    end
    #save plane solution to filepath
    plane_sol = LTB4_relay_inhib_w_decay_2d3d_sol[translate_2d3d_to_1d.(1:N_r,ceil(Int,N_z/2),N_r),:,:] #Get the solution at the z=0 plane

    reshaped_plane_sol = reshape(plane_sol, (size(plane_sol, 1)*size(plane_sol, 2) , size(plane_sol, 3))) #Reshape the solution to be a 2D array
    sol_df = DataFrame(reshaped_plane_sol, :auto)
    rename!(sol_df, [Symbol("ti=$(i)") for i in 1:size(sol_df, 2)])
    CSV.write(filepath , sol_df)
    #save parameters to params_filepath
    params_dict = Dict("D" => D, "a" => a, "gammaL" => gammaL, "L0" => L0, "L1" => L1, 
        "b" => b, "I" => I, "rho" => rho, "S_on" => S_on, 
        "S_duration" => S_duration, "s_init" => s_init, "r_source" => r_source ,  "model" => model, 
        "dr" =>dr, "dt" =>savedt, "t_mul"=>t_mul,"l_mul"=>l_mul)
    if model == "practical"
        params_dict["gammaR"] = gammaR
    end

    open(params_filepath, "w") do file
        write(file, JSON.json(params_dict))
    end
  end

In [ ]:
l_mul = 10
t_mul = 100
r_limit = Float32(50*l_mul) # unit: micron, seconds
z_limit = Float32(5*l_mul) # unit: micron

# dr = step(rd_LTB4);
# dz = step(zd_LTB4);

D = 0.125*1e3*l_mul^2 /t_mul #Diffusion constant
#Cond set:
b = 1/(t_mul) #Inhibition rate 
I = 0.365#Inhibition shutoff threshold
gammaL = 0.2/t_mul #Decay rate #gammaL 0.1
gammaR = 0.1/t_mul#Decay rate #gammaR 0.1
#Units are scaled but otherwise set here: 
a = 1/t_mul #Relay strength term, effectively (a*p from paul model) a = 0.025*t_mul*l_mul^2 

sam = 9.5e3 #Input source strength, without units, 0 means no input after initialization
S_duration = 0.5*t_mul #Duration of input source

rho = 1.0/l_mul^2 #2D density of cells
S_on = sam/(t_mul^2*l_mul^3) #Input strength, constant emmisssion, with units
L0 = 0.0074/l_mul^3 #Relay threshold

#Inhibition Terms: 
L1 = L0/100 #Inhibitor production threshold

#Parameter set to pass to model
ps_temp = D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_duration #For LTB4_relay_gdecay_practical_2d3d_diff


In [ ]:
#study the effect of relay strength,activator threshold and prohibitor threshold on the output:
a_s = [1.0]./t_mul
# L0_s = Array(0.005:0.0001:0.009) ./l_mul^3 
L0 = 0.007/l_mul^3
L1_s = L0./(1:2:100)
I_s = 0.1:0.03:1.0
parameter_screen = [] #store all parameter sets
for a in a_s
    for L1 in L1_s
        for I in I_s
            #Now a relay that has a shutoff counter inhibition going on activated at a higher threshold: 
            
            # L1 = L0/100
            
            # S_on = 0.
            # a = 1.0
            #Parameter set to pass to model
            ps_temp =D, a, gammaL, gammaR, L0, L1, b, I, rho, S_on, S_duration,l_mul,t_mul

            push!(parameter_screen, ps_temp)
        end
    end
end

In [ ]:
using Base.Threads
t0 = time()
N = length(parameter_screen)
paras_idx = collect(1:N)
@threads for i in paras_idx #use multi-threads to accelerate the simulation
    ps = parameter_screen[i]
    run_ODE_Solve(ps, ps_init, model = "practical", folder = "pulse parameter screen practical 335")
    # i += 1
    if mod(i,10) == 0
        t1 = time()
        println("$(i) finished")
    end
end


# Analyze the results

In [ ]:
using StatsBase

function wave_front_r(plane_sol_at_t, dr, S_cri)
    # Find the wave front position where the concentration is larger than S_cri
    N_r = Int(length(plane_sol_at_t)/2)
    ix = findlast(plane_sol_at_t[1:N_r] .> S_cri)
    if ix == nothing
        return 0
    else
        return ix*dr
    end
end

function analyze_wavespeed(filepath)
    #Find the wave speed from the wavefront position, which should be the largest value in the wavespeed histogram
    # Read the solution from the CSV file
    sol_df = CSV.read(string(filepath,".csv"), DataFrame)
    params = JSON.parsefile(string(filepath,"_params.json"))
    N_t = size(sol_df, 2)
    L1 = params["L1"]
    L0 = params["L0"]
    dr = params["dr"]
    dt = params["dt"]
    try # in case some simulation does not run successfully
        r_mid = [wave_front_r(sol_df[:,ti], dr , sqrt(L1*L0)) for ti in 1:N_t]
        vs = diff(r_mid)./dt
        vs = Array(vs[vs .> 0])
        # nbins = 50
        hist = fit(Histogram, vs, nbins=50)
        max_bin_index = findmax(hist.weights)[2]
        wave_speed = (hist.edges[1][max_bin_index] + hist.edges[1][max_bin_index+1])/2

        return wave_speed
    catch e
        return 0.
    end
end

function analyze_maxr(filepath)
    # Find the maximum radius of the wavefront
    # Read the solution from the CSV file
    sol_df = CSV.read(string(filepath,".csv"), DataFrame)
    params = JSON.parsefile(string(filepath,"_params.json"))
    N_t = size(sol_df, 2)
    L1 = params["L1"]
    L0 = params["L0"]
    dr = params["dr"]
    r_mid = [wave_front_r(sol_df[:,ti], dr , sqrt(L0*L1 )) for ti in 1:N_t]
    max_r = maximum(r_mid)

    return max_r
end

In [ ]:
function analyze_solutions(folder,a_s, L0_s, inhib_shutoff_s)
    # Find all variables of interest from the solutions
    max_rs = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    wave_speeds = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value1 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value2 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value3 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value4 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value5 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    char_value6 = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    for i in 1:length(a_s)
        for j in 1:length(L0_s)
            for k in 1:length(inhib_shutoff_s)
                L0 = L0_s[j]
                a = a_s[i]
                L1 = L0 / 100
                inhib_shutoff = inhib_shutoff_s[k]

                # filename = string("Sa-", round(s_init,digits = 2),"_a-",round(a,digits = 2),"_L1a-", 
                # round(L0/(a*rho),digits = 5), "_L1L2-", round(L0/L1,digits=2), "_R1b-", round(inhib_shutoff/inhib_rate,digits=2))
                # filepath = joinpath(folder, filename) #naming for Gaussian initial source
                filename = string("Son-", round(S_on,sigdigits = 3),"_a-",round(a,sigdigits = 3),"_L1a-", round(L0/(a*rho),sigdigits = 3),
                "_L1L2-", round(L0/L1,sigdigits=3), "_R1b-", round(inhib_shutoff/inhib_rate,sigdigits=3)) #naming for pulsatile source
                filepath = joinpath(folder, filename)
                if isfile(string(filepath,".csv"))
                    max_rs[i,j,k] = analyze_maxr(filepath)
                    wave_speeds[i,j,k] = analyze_wavespeed(filepath)
                    char_value1[i,j,k] = 2*a*rho/(pi*L0) #theoretical wave speed
                    char_value2[i,j,k] = 4*inhib_shutoff*a^2*rho^2/(pi^2*inhib_rate*DL*L0^2) #theoretical wave-stop characteristic value
                    char_value3[i,j,k] = inhib_shutoff/inhib_rate*wave_speeds[i,j,k]^2/DL #semi-theoretical wave-stop characteristic value
                    char_value4[i,j,k] = inhib_shutoff/inhib_rate*(2*a*rho/(pi*L0)) #theoretical wave-stop radius
                    char_value5[i,j,k] = inhib_shutoff/inhib_rate*wave_speeds[i,j,k] #semi-theoretical wave-stop radius
                    char_value6[i,j,k] = sqrt(4*DL*gammaL)*(1/sin(L0*pi*sqrt(DL*gammaL)/(a*rho))^2-1) #theoretical wave speed with decay term
                else
                    println("File not found: ", filepath)
                end
            end
        end
    end

    return max_rs,wave_speeds,char_value1, char_value2,char_value3, char_value4, char_value5,char_value6
end

function check_file(folder, a_s, L0_s, inhib_shutoff_s)
    # Check if the file exists
    is_file_mat = zeros(length(a_s),length(L0_s), length(inhib_shutoff_s))
    for i in 1:length(a_s)
        for j in 1:length(L0_s)
            for k in 1:length(inhib_shutoff_s)
                L0 = L0_s[j]
                a = a_s[i]
                L1 = L0 / 100
                inhib_shutoff = inhib_shutoff_s[k]

                # filename = string("Sa-", round(s_init,digits = 2),"_a-",round(a,digits = 2),"_L1a-", 
                # round(L0/(a*rho),digits = 5), "_L1L2-", round(L0/L1,digits=2), "_R1b-", round(inhib_shutoff/inhib_rate,digits=2))
                # filepath = joinpath(folder, filename) #naming for Gaussian initial source
                filename = string("Son-", round(S_on,sigdigits = 3),"_a-",round(a,sigdigits = 3),"_L1a-", round(L0/(a*rho),sigdigits = 3),
                "_L1L2-", round(L0/L1,sigdigits=3), "_R1b-", round(inhib_shutoff/inhib_rate,sigdigits=3)) #naming for pulsatile source
                filepath = joinpath(folder, filename)
                if isfile(string(filepath,".csv"))
                    is_file_mat[i,j,k] = 1
                else
                    is_file_mat[i,j,k] = 0
                end
            end
        end
    end
    return is_file_mat
end
#We have run different parameter sets for different mnk(hill coefficient) values
#parameter set 1, good for studying relay wave speed since many relaying case in this set, mnk=335
# a_s = [1.0]./t_mul
# L0_s = [0.1,0.05,0.01,0.009,0.0074,0.006,0.005,0.003,0.001]./l_mul^3
# inhib_shutoff_s = [3.,1.5,1.2,1.0,0.8,0.5,0.3]

#parameter set 2, compared with set 1, set 2 is narrower in L0 since diffusion dominates for large L0 in set 1 and relay dominates for small L0 in set 1, good for studying wave stop characteristic value and initial condition restriction, mnk=335
# a_s = [1.0]./t_mul
# L0_s = 10 .^(-2.2:0.05:-1.7) ./l_mul^3
# inhib_shutoff_s = [3.,1.5,1.2,1.0,0.8,0.5,0.3]

#parameter set 3, good for studying wave stop characteristic value since it focus on wave-stop cases, mnk=335
# a_s = [1.0]./t_mul
# L0_s = 10 .^(-2.2:0.02:-2.1) ./l_mul^3
# inhib_shutoff_s = [1.4,1.3,1.2,1.1,1.0,0.9,0.8,0.7]

#parameter set 4, mnk = 101020
# a_s = [1.0]./t_mul
# L0_s = [0.1,0.05,0.01,0.009,0.0074,0.006,0.005,0.003,0.001]./l_mul^3
# inhib_shutoff_s = [0.1,0.2,0.3,0.32,0.34,0.36,0.365,0.38,0.4,0.5,0.6]

#parameter set 5, mnk = 101020, producing uniform L1 and R1 screen
# a_s = [1.0]./t_mul
# L0_s = Array(0.001:0.001:0.02) ./l_mul^3
# inhib_shutoff_s = 0.1:0.1:1

#parameter set 6, mnk = 335, producing uniform L1 and R1 screen
# a_s = [1.0]./t_mul
# L0_s = Array(0.001:0.001:0.02) ./l_mul^3
# inhib_shutoff_s = 0.5:0.1:1.5

#parameter set 7, mnk = 335 and 101020, producing uniform L1 and R1 screen
# a_s = [1.0]./t_mul
# L0_s = Array(0.005:0.0003:0.01) ./l_mul^3
# inhib_shutoff_s = 0.1:0.1:1.0

#parameter set 8, mnk = 335 and 101020, producing uniform L1 and R1 screen,it could take 5 mins to process, I save the processed data in JLD file under the folder
a_s = [1.0]./t_mul
L0_s = Array(0.005:0.0001:0.009) ./l_mul^3
inhib_shutoff_s = 0.1:0.01:1.0

check_file_mat = check_file("pulse parameter screen practical 335", a_s, L0_s, inhib_shutoff_s);
max_rs,wave_speeds,theory_wave_speeds, theory_char_values, semi_theory_char_values, theory_radii, semi_theory_radii, wave_speeds_wdecay= analyze_solutions("pulse parameter screen practical 335",a_s, L0_s, inhib_shutoff_s);


In [ ]:
# Plot the heatmap of the maximum radius, with respect to different parameters
using ImageFiltering #for smoothing the discrete data
p1=heatmap(L0_s,inhib_shutoff_s,max_rs[1,:,:]',colormap=:RdBu,α=1,clims=(0,500),colorbar_title=L"r_m/\mu m", xformatter = x->string(round(Int, x / 10^-6)))
# annotate!([(1.02*maximum(L0_s),0.05, Plots.text(L"\times10^{-6}", 11, :black, :center))])
z=reshape(log10.(vec(semi_theory_char_values)),(length(L0_s),length(inhib_shutoff_s)))'
smooth_z = imfilter(z, Kernel.gaussian(1.5))
beta_line = 6.3
contour!(p1,L0_s,inhib_shutoff_s,smooth_z,levels=[log10(beta_line)],clabels=false,lw=2,label="beta=$(beta_line)",line_smoothing = 2.8) #Plot the beta line to compare that with the maximum radius
annotate!(p1,7e-6,0.4,text(L"\beta ="*"$(beta_line)",12,:white))
plot!(p1,xlabel=L"L_0\times 10^6",ylabel=L"I\times 10^6",title=L"\beta"*" dependence")